In [5]:
# set up environment
import warnings;
warnings.filterwarnings('ignore');
import pandas as pd
import os.path as op
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import numpy as np
InteractiveShell.ast_node_interactivity = "all"
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
import matplotlib.pyplot as plt;
from matplotlib.lines import Line2D;
import sys
import scipy.stats
import os
from matplotlib import cm
from matplotlib.colors import Normalize
cmap = cm.get_cmap('magma')
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib.lines import Line2D;
from sklearn.metrics import r2_score
import scipy
from scipy.stats import mannwhitneyu

In [6]:
# Get data
figshare_url = "https://figshare.com/ndownloader/files/"
file_dict = {"dHCP_sl_count.csv": "41514570", 
             "matched_data_final.csv": "41514573", 
             "participants.csv": "41514576",
             "QC_combined.csv": "41514579", 
             "sessions.csv": "41514582",
             "tract_profiles_with_t1wt2w_nonorm.csv": "41514585"}


import urllib.request
for kk, vv in file_dict.items():
    urllib.request.urlretrieve(figshare_url + vv, "inputData/" + kk)


('inputData/dHCP_sl_count.csv', <http.client.HTTPMessage at 0x7fd140ebde80>)

('inputData/matched_data_final.csv',
 <http.client.HTTPMessage at 0x7fd140ebd760>)

('inputData/participants.csv', <http.client.HTTPMessage at 0x7fd140ebd5b0>)

('inputData/QC_combined.csv', <http.client.HTTPMessage at 0x7fd140ebd7c0>)

('inputData/sessions.csv', <http.client.HTTPMessage at 0x7fd140ebd820>)

('inputData/tract_profiles_with_t1wt2w_nonorm.csv',
 <http.client.HTTPMessage at 0x7fd140ebd700>)

In [ ]:
#plotting parameters, these will the same for all plots

x_labels=['Node (pos -> ant)', 'Node (pos -> ant)',
          'Node (pos -> ant)','Node (pos -> ant)',
          'Node (pos -> ant)', 'Node (pos -> ant)',
          'Node (inf -> sup)', 'Node (inf -> sup)',
          'Node (lh -> rh)', 'Node (lh -> rh)',
          'Node (pos -> ant)','Node (pos -> ant)',
          'Node (pos -> ant)', 'Node (pos -> ant)',
          'Node (pos -> ant)', 'Node (pos -> ant)',
          'Node (pos -> ant)', 'Node (pos -> ant)',
          'Node (pos -> ant)', 'Node (pos -> ant)'];

tracts=['AF', 'AF', 'ATR', 'ATR', 'CC', 'CC',
        'CS', 'CS', 'FcMi', 'FcMa','IFOF', 'IFOF',
        'ILF', 'ILF', 'MLF', 'MLF','SLF', 'SLF', 
        'UNC', 'UNC'];

tractPos = {'ATR': (0, 0), 'CS': (0, 1), 'CC': (0, 2), 'FcMi': (0, 4), 'FcMa': (0, 5),
            'IFOF': (1, 2), 'ILF':(1,0), 'MLF':(1,1), 'SLF':(1,3), 'UNC':(1,4), 'AF': (1, 5)};

tractPosHorz = {'ATR': (0, 0), 'CS': (0, 1), 'CC': (0, 2), 
            'ILF':(1,0), 'MLF':(1,1), 'SLF':(1,2),
            'IFOF': (2, 0),  'UNC':(2,1), 'AF': (2, 2), 
            'FcMi': (3, 0), 'FcMa': (3, 1),};

color_list_all=sns.color_palette("tab20")+sns.color_palette("tab20b") 
color_list_complete=color_list_all
color_list_chosen=color_list_all[0:14]
color_list_chosen.extend(color_list_all[24:25])
color_list_chosen.extend(color_list_all[27:28])
color_list_chosen.extend(color_list_all[16:20]) 
sns.palplot(color_list_all)
sns.palplot(color_list_chosen)
color_list_all=color_list_chosen
color_order=[19, 18, 1, 0, 5, 4, 3, 2, 7, 6, 9, 8, 11, 10, 15, 14, 13, 12, 17, 16]

tractsRh=['ARC_R', 'ATR_R', 'CGC_R', 'CST_R', 'IFO_R', 'ILF_R', 'MdLF_R', 'SLF_R', 'UNC_R']
tractNrRh=[0,2,4,6,10,12,14,16,18]
tractsLh=['ARC_L', 'ATR_L', 'CGC_L', 'CST_L', 'IFO_L', 'ILF_L', 'MdLF_L', 'SLF_L', 'UNC_L']
tractNrLh=[1,3,5,7,11,13,15,17,19]



In [ ]:
#load all data from local folder

# load participant information from local folder
parts = pd.read_csv('./inputData/participants.csv', index_col=0)

#load QC data from local folder
QC = pd.read_csv('./inputData/QC_combined.csv', index_col=0)

#load session information from local folder
sessions = pd.read_csv('./inputData/sessions.csv')

sessions.rename(columns={"Unnamed: 0": "session_index"}, inplace=True)

#load tract profiles from local folder
tract_profiles = pd.read_csv("./inputData/tract_profiles_with_t1wt2w_nonorm.csv", index_col=0)


In [ ]:
#perform quality control

#merge session and QC data, set QC treshhold

sessionsQC = pd.merge(QC, sessions, on=["session_id","participant_id"], how="inner")
sessionsQC.shape
#sessionsQC.head(20)
sessionsQC["qc_dmri_eddy_abs_motion"].mean()
sessionsQC["qc_dmri_eddy_abs_motion"].std()
motion_cutoff=sessionsQC["qc_dmri_eddy_abs_motion"].mean()+2*sessionsQC["qc_dmri_eddy_abs_motion"].std()
print(motion_cutoff)
sessionsQC["qc_dmri_eddy_percent_outlier_slices"].mean()
sessionsQC["qc_dmri_eddy_percent_outlier_slices"].std()
outlier_cutoff=sessionsQC["qc_dmri_eddy_percent_outlier_slices"].mean()+2*sessionsQC["qc_dmri_eddy_percent_outlier_slices"].std()
print(outlier_cutoff)

sessions_afterQC=sessionsQC[sessionsQC["qc_dmri_eddy_abs_motion"]<motion_cutoff]
sessions_afterQC=sessions_afterQC[sessions_afterQC["qc_dmri_eddy_percent_outlier_slices"]<outlier_cutoff]
sessions_afterQC.shape
sessionsAfterQC=sessions[sessions["session_id"].isin(sessions_afterQC["session_id"])]
sessionsAfterQC.shape
sessionsAfterQC=sessionsAfterQC[sessionsAfterQC['participant_id']!='CC00530XX11'];
sessionsAfterQC=sessionsAfterQC[sessionsAfterQC['participant_id']!='CC00129AN14'];
sessionsAfterQC=sessionsAfterQC[sessionsAfterQC['participant_id']!='CC00822XX14'];
sessionsAfterQC=sessionsAfterQC[sessionsAfterQC['participant_id']!='CC00766XX16'];
sessionsAfterQC=sessionsAfterQC[sessionsAfterQC['participant_id']!='CC00572CN12'];
sessionsAfterQC.shape

# load sl count for local storage and remove bundles with fever than 11 streamlines
subjects = pd.merge(parts, sessionsAfterQC, on="participant_id", how="inner")
subjects=subjects[subjects["session_id"].isin(tract_profiles["sessionID"])]

subjects.rename(columns={"participant_id": "subjectID"}, inplace=True)
subjects.rename(columns={"session_id": "sessionID"}, inplace=True)
os.makedirs('./outputData', exist_ok=True) 
subjects.to_csv("./outputData/subjects.csv")
#subjects.head()

allDataMerged=pd.merge(tract_profiles,subjects,on=['subjectID','sessionID'])
allDataMerged.head()
allDataMerged.shape

sl_count_all=pd.read_csv("./inputData/dHCP_sl_count.csv")
low_sl_count=sl_count_all[sl_count_all['n_streamlines_clean']<=10]['subject_id'].unique()
low_sl_count.shape

allDataCleaned=allDataMerged[~allDataMerged["subjectID"].isin(low_sl_count)]
allDataCleaned.shape

subjects.shape
subjectsCleaned=subjects[~subjects["subjectID"].isin(low_sl_count)]
subjectsCleaned.shape

tract_profilesCleaned=tract_profiles[~tract_profiles["subjectID"].isin(low_sl_count)]
#tract_profilesCleaned.shape

sessionsAfterQC.shape
sessionsCleaned=sessionsAfterQC[~sessionsAfterQC["participant_id"].isin(low_sl_count)]
sessionsCleaned.shape


In [ ]:
#identify longi subjects
subjectsCleaned.shape
subjectsCLeanedLongi=subjectsCleaned[subjectsCleaned.duplicated(subset="subjectID")]
subjectsCLeanedLongi.shape

subjectsCleaned["isLongi"]=subjectsCleaned["subjectID"].isin(subjectsCLeanedLongi["subjectID"])
subjectsCleaned["isLongi"]=subjectsCleaned["isLongi"].astype(int)
subjectsCleaned["timeAfterBirth"]=subjectsCleaned['scan_age']-subjectsCleaned['birth_age']

subjectsCLeanedNoLongi=subjectsCleaned[~subjectsCleaned.duplicated(subset="subjectID")]
subjectsCLeanedNoLongi.shape
subjectsCLeanedNoLongi["timeAfterBirth"].mean()
subjectsCLeanedNoLongi["timeAfterBirth"].min()
subjectsCLeanedNoLongi["timeAfterBirth"].max()

subjectsCLeanedNoLongi["birth_age"].min()
subjectsCLeanedNoLongi["birth_age"].max()
#subjectsCleaned["subjectID"].unique
#subjects.head()
#subjects['scan_age'].hist()
#subjects['birth_age'].hist()

subjectsCleaned["scan_age"].max()


In [ ]:
# assign matched labels to subjects table and all data table
#load the data that was matched sucessfully

matchedData=pd.read_csv('./inputData/matched_data_final.csv') #this table includes the single matches fro pymatch
#and Ariels nearest neighbor

subjectsCleaned["matched"]=0
#subjectsCleaned.head()
subjectsCleaned["matched"]=subjectsCleaned['subjectID'].isin(matchedData['subjectID']).astype(int)

subjectsCleaned["matched_groups"]=0
subjectsCleaned["matched_groups"][(subjectsCleaned["isLongi"]>0) & (subjectsCleaned['matched']>0) & (subjectsCleaned['scan_number']==1)]=1
subjectsCleaned["matched_groups"][subjectsCleaned["matched_groups"]==1].shape
subjectsCleaned["matched_groups"][(subjectsCleaned["isLongi"]>0) & (subjectsCleaned['matched']>0) & (subjectsCleaned['scan_number']==2)]=2
subjectsCleaned["matched_groups"][subjectsCleaned["matched_groups"]==2].shape
subjectsCleaned["matched_groups"][(subjectsCleaned["isLongi"]==0) & (subjectsCleaned['matched']>0)]=3
subjectsCleaned["matched_groups"][subjectsCleaned["matched_groups"]==3].shape
#subjectsCleaned.head()

subjectsCleaned[subjectsCleaned['matched_groups']==0].shape
subjectsCleaned[subjectsCleaned['matched_groups']==1].shape
subjectsCleaned[subjectsCleaned['matched_groups']==2].shape
subjectsCleaned[subjectsCleaned['matched_groups']==3].shape

subjectsCleanedMatched=subjectsCleaned[subjectsCleaned['matched_groups']>0]
age0=subjectsCleaned[subjectsCleaned['matched_groups']==1]
age0['scan_age'].mean()
age0.shape
age1=subjectsCleaned[subjectsCleaned['matched_groups']==2]
age1['scan_age'].mean()
age1.shape
age1['scan_age'].min()

age2=subjectsCleaned[subjectsCleaned['matched_groups']==3]
age2['scan_age'].mean()
age2.shape
age0['birth_age'].hist()

age3=subjectsCleaned[subjectsCleaned['matched_groups']==1]
age3['birth_age'].mean()
age3.shape
age4=subjectsCleaned[subjectsCleaned['matched_groups']==2]
age4['birth_age'].mean()
age4.shape
age5=subjectsCleaned[subjectsCleaned['matched_groups']==3]
age5['birth_age'].mean()
age5.shape

subjectsCleanedNonMatched=subjectsCleaned[subjectsCleaned['matched_groups']==0]
subjectsCleanedNonMatched['timeAfterBirth'].min()
subjectsCleanedNonMatched['timeAfterBirth'].max()
subjectsCleanedNonMatched['timeAfterBirth'].mean()
sns.histplot(subjectsCleanedNonMatched['timeAfterBirth'])


allDataMergedCleaned=pd.merge(tract_profilesCleaned,subjectsCleaned,on=['subjectID','sessionID'])
#allDataMergedCleaned.head(20)
allDataMerged.shape
allDataCleaned.shape
allDataMergedCleaned.shape

allDataMergedCleaned[allDataMergedCleaned['matched_groups']==0].shape
allDataMergedCleaned[allDataMergedCleaned['matched_groups']==1].shape
allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2].shape
allDataMergedCleaned[allDataMergedCleaned['matched_groups']==3].shape

tractCount=allDataMergedCleaned['tractID'].unique()
tractCount.shape
nodeCount=allDataMergedCleaned['nodeID'].unique()
nodeCount.shape
groupCount=allDataMergedCleaned['matched_groups'].unique()
groupCount.shape

print(tractCount)
allDataMergedCleaned.head()
allDataMergedCleaned['dti_md_x100']=allDataMergedCleaned['dti_md']*100


In [ ]:
# this generates fig 1 in the main manuscript
#now we are looking at the non-longi sample
#here we create a simple plot of dev over time
allDataCleanedNoLongi=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==0]
allDataCleanedNoLongiIU=allDataCleanedNoLongi[allDataCleanedNoLongi["timeAfterBirth"]<=4]
allDataCleanedNoLongiIU['ageBin']=0
allDataCleanedNoLongiIU['ageBin'][(allDataCleanedNoLongiIU['scan_age']>=33) & (allDataCleanedNoLongiIU['scan_age']<37)]=33
allDataCleanedNoLongiIU['ageBin'][(allDataCleanedNoLongiIU['scan_age']>=37) & (allDataCleanedNoLongiIU['scan_age']<41)]=37
allDataCleanedNoLongiIU['ageBin'][(allDataCleanedNoLongiIU['scan_age']>=41) & (allDataCleanedNoLongiIU['scan_age']<45)]=41
#...
allDataCleanedNoLongiIUBinned=allDataCleanedNoLongiIU[allDataCleanedNoLongiIU['ageBin']>0]
df= allDataCleanedNoLongiIUBinned;

fig, axes = plt.subplots(4,3,frameon=False,figsize=(15,23));
fig.tight_layout;
fig.subplots_adjust(wspace=0.3, hspace=0.9);
sns.set(font_scale = 2);
sns.set_style('white');

ct = 0;
for tractName in tractCount:
    ax=axes[tractPosHorz[tracts[ct]]];
    _=ax.set_ylim([0.3,2.2])
    currentTract=df.query("tractID == @tractName");
    sns.set_style('white');
    
    bin=33
    currentBin=currentTract.query("ageBin == @bin");
    fig1=sns.lineplot(data=currentBin, x='nodeID', y='t1wt2w',hue='ageBin' ,legend=False, style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    bin=37
    currentBin=currentTract.query("ageBin == @bin");
    fig1=sns.lineplot(data=currentBin, x='nodeID', y='t1wt2w',hue='ageBin' ,legend=False, dashes=[(3,3)], style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    bin=41
    currentBin=currentTract.query("ageBin == @bin");
    fig1=sns.lineplot(data=currentBin, x='nodeID', y='t1wt2w',hue='ageBin' ,legend=False, dashes=[(1,1)], style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    
    _= ax.set_title(tracts[ct], pad=25, fontsize=25);
    _ = ax.set_xlabel(x_labels[ct], labelpad = 15, fontsize=25);
    
    if ct==2 or ct==3 or ct==12 or ct==13 or ct==8 or ct==10 or ct==11:
        _=ax.set_ylabel("T1w/T2w",fontsize=25);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,-1.0,-0.5,0.0])
        #ax.set_yticklabels([-1.5,-1.0,-0.5,0.0],fontsize=25)
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['top'].set_visible(False);
        _=line1=Line2D([],[],color=color_list_all[ct],linestyle='dashed');
        _=line2=Line2D([],[],color=color_list_all[ct],linestyle='-');
        #ax.legend([line1,line2],['BB','AB'],loc="upper left",borderpad=0.01,bbox_to_anchor=(0.15,0.2),ncol=2,framealpha=0.75,frameon=True,edgecolor=[1,1,1],prop={'size':20},fancybox=True,handlelength=1.5)
        
    else:
        _=ax.set_ylabel('', fontsize=14);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,0.0])
        _=ax.yaxis.set_visible(False);
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['left'].set_visible(False);
        _=ax.spines['top'].set_visible(False);

    ct = ct + 1;
    
axes[3,2].axis("off");
sns.set_style('white');
plt.show();
os.makedirs('./figures',exist_ok=True)
fig.savefig('./figures/fig1_sample1_different_scan_ages.png', dpi=600)



In [ ]:
#compute slope for scan age along bundles

allDataCleanedNoLongi=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==0]
tractCount.shape
coeff=[] #Creating arrays
se = []
coeffType=[]
tractTrack1=[]
nodeTrack1=[]
intercept=[]
r2 = []
adjustR2 = []
aic = []
pvals=[]
    
for tract in tractCount:
        
    for node in nodeCount:
        
        dfForStats=allDataCleanedNoLongi[(allDataCleanedNoLongi['tractID']==tract) & (allDataCleanedNoLongi['nodeID']==node)]
        dfForStats.reset_index(level=0, inplace=True)

        scanAgeArray=np.array(dfForStats['scan_age'])
        scanAgeArray=scanAgeArray.reshape(1,-1)
        scanAgeNorm=preprocessing.normalize(scanAgeArray,axis=1)
        scanAgeNorm=scanAgeNorm.reshape(-1)
        dfForStats["scanAgeNorm"]=scanAgeNorm

        birthAgeArray=np.array(dfForStats['birth_age'])
        birthAgeArray=birthAgeArray.reshape(1,-1)
        birthAgeNorm=preprocessing.normalize(birthAgeArray,axis=1)
        birthAgeNorm=birthAgeNorm.reshape(-1)
        dfForStats["birthAgeNorm"]=birthAgeNorm

        timeAfterBirthArray=np.array(dfForStats['timeAfterBirth'])
        timeAfterBirthArray=timeAfterBirthArray.reshape(1,-1)
        timeAfterBirthNorm=preprocessing.normalize(timeAfterBirthArray,axis=1)
        timeAfterBirthNorm=timeAfterBirthNorm.reshape(-1)
        dfForStats["timeAfterBirthNorm"]=timeAfterBirthNorm

        dti_mdArray=np.array(dfForStats['dti_md_x100'])
        dti_mdArray=dti_mdArray.reshape(1,-1)
        dti_mdNorm=preprocessing.normalize(dti_mdArray,axis=1)
        dti_mdNorm=dti_mdNorm.reshape(-1)
        dfForStats["dti_mdNorm"]=dti_mdNorm
        
        dti_mdArray=np.array(dfForStats['dti_fa'])
        dti_mdArray=dti_mdArray.reshape(1,-1)
        dti_mdNorm=preprocessing.normalize(dti_mdArray,axis=1)
        dti_mdNorm=dti_mdNorm.reshape(-1)
        dfForStats["dti_faNorm"]=dti_mdNorm
        
        t1wt2wArray=np.array(dfForStats['t1wt2w'])
        t1wt2wArray=t1wt2wArray.reshape(1,-1)
        t1wt2wNorm=preprocessing.normalize(t1wt2wArray,axis=1)
        t1wt2wNorm=t1wt2wNorm.reshape(-1)
        dfForStats["t1wt2wNorm"]=t1wt2wNorm
            #dfForStats.head()

            #dfForStats.to_csv('./'+tract+'_dataCleanedForLME.csv')
        df = dfForStats
        md = smf.ols(formula="t1wt2w~ 1 + scan_age", data=df) #Predicting dti_mdNorm through the other variables
        mdf = md.fit() # Actually commanding the linear analysis, it could be any other analysis method
        results=mdf.summary()

        # Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
        results_as_html = results.tables[1].as_html()
        LM_results=pd.read_html(results_as_html, header=0, index_col=0)[0]
        
        results_as_html_R2 = results.tables[0].as_html()
        LM_results_R2=pd.read_html(results_as_html_R2, index_col=0)[0]
        
        results_R2s=LM_results_R2[3]
        #LM_results.head()
        #LM_results_R2.head()
        #results_R2s.head()
        
        SE=LM_results['std err']
        
        inter=mdf.params['Intercept']
        intercept.append(inter)
        
        #SE.head()
        se.append(SE["scan_age"])
        coeff.append(mdf.params['scan_age'])
        coeff_type=3
        tractTrack1.append(tract)
        nodeTrack1.append(node)
        coeffType.append(coeff_type) #Add another line below
        r2.append(results_R2s['Dep. Variable:'])
        adjustR2.append(results_R2s['Model:'])
        aic.append(results_R2s['No. Observations:']) 
        pvals.append(results_R2s['Date:'])        
        
print(mdf.summary())
df=pd.DataFrame(tractTrack1)
df.columns=['tract']
df.insert(1,"node",nodeTrack1)
df.insert(2,"coeffType",coeffType)
df.insert(3,"coeff",coeff)
df.insert(4,"se",se)
df.insert(5,"intercept",intercept)
df.insert(6,"r2",r2)
df.insert(7,"adjustR2",adjustR2)
df.insert(8,"aic",aic)
df.insert(9,"pvals",pvals)

df.to_csv('./outputData/LM_result_scanAge_noNorm.csv')
df.head()
df.min(axis=0, numeric_only=True)
df.max(axis=0, numeric_only=True)
df.mean(axis=0, numeric_only=True)
dfScanAge=df
df.head()

statsPerBundle=df.groupby('tract').mean()
statsPerBundle.head()
statsPerBundle.to_csv('./outputData/LM_result_scanAge_noNorm_meanPerBundle.csv')


In [ ]:
#compute slope for birth age and time after birth along bundles, no normalization

allDataCleanedNoLongi=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==0]
tractCount.shape
coeff=[] #Creating arrays
se = []
coeffType=[]
tractTrack1=[]
nodeTrack1=[]
intercept=[]
r2 = []
adjustR2 = []
aic = []
pvals=[]
    
for tract in tractCount:
        
    for node in nodeCount:
        
        dfForStats=allDataCleanedNoLongi[(allDataCleanedNoLongi['tractID']==tract) & (allDataCleanedNoLongi['nodeID']==node)]
        dfForStats.reset_index(level=0, inplace=True)

        scanAgeArray=np.array(dfForStats['scan_age'])
        scanAgeArray=scanAgeArray.reshape(1,-1)
        scanAgeNorm=preprocessing.normalize(scanAgeArray,axis=1)
        scanAgeNorm=scanAgeNorm.reshape(-1)
        dfForStats["scanAgeNorm"]=scanAgeNorm

        birthAgeArray=np.array(dfForStats['birth_age'])
        birthAgeArray=birthAgeArray.reshape(1,-1)
        birthAgeNorm=preprocessing.normalize(birthAgeArray,axis=1)
        birthAgeNorm=birthAgeNorm.reshape(-1)
        dfForStats["birthAgeNorm"]=birthAgeNorm

        timeAfterBirthArray=np.array(dfForStats['timeAfterBirth'])
        timeAfterBirthArray=timeAfterBirthArray.reshape(1,-1)
        timeAfterBirthNorm=preprocessing.normalize(timeAfterBirthArray,axis=1)
        timeAfterBirthNorm=timeAfterBirthNorm.reshape(-1)
        dfForStats["timeAfterBirthNorm"]=timeAfterBirthNorm

        dti_mdArray=np.array(dfForStats['dti_md_x100'])
        dti_mdArray=dti_mdArray.reshape(1,-1)
        dti_mdNorm=preprocessing.normalize(dti_mdArray,axis=1)
        dti_mdNorm=dti_mdNorm.reshape(-1)
        dfForStats["dti_mdNorm"]=dti_mdNorm
        
        dti_mdArray=np.array(dfForStats['dti_fa'])
        dti_mdArray=dti_mdArray.reshape(1,-1)
        dti_mdNorm=preprocessing.normalize(dti_mdArray,axis=1)
        dti_mdNorm=dti_mdNorm.reshape(-1)
        dfForStats["dti_faNorm"]=dti_mdNorm
        
        t1wt2wArray=np.array(dfForStats['t1wt2w'])
        t1wt2wArray=t1wt2wArray.reshape(1,-1)
        t1wt2wNorm=preprocessing.normalize(t1wt2wArray,axis=1)
        t1wt2wNorm=t1wt2wNorm.reshape(-1)
        dfForStats["t1wt2wNorm"]=t1wt2wNorm
            #dfForStats.head()

            #dfForStats.to_csv('./'+tract+'_dataCleanedForLME.csv'
        df = dfForStats
        md = smf.ols(formula="t1wt2w ~ 1 + birth_age + timeAfterBirth", data=df) #Predicting dti_mdNorm through the other variables
        mdf = md.fit() # Actually commanding the linear analysis, it could be any other analysis method
        results=mdf.summary()

# Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
        results_as_html = results.tables[1].as_html()
        LM_results=pd.read_html(results_as_html, header=0, index_col=0)[0]
        
        results_as_html_R2 = results.tables[0].as_html()
        LM_results_R2=pd.read_html(results_as_html_R2, index_col=0)[0]     
        results_R2s=LM_results_R2[3]
        
        coeff_type = 1
        #LM_results.head()
        SE=LM_results['std err']
        inter=mdf.params['Intercept']
        intercept.append(inter)
        se.append(SE["birth_age"])
        coeff.append(mdf.params['birth_age'])
        tractTrack1.append(tract)
        nodeTrack1.append(node)
        coeffType.append(coeff_type) #Add another line below
        
        coeff_type=2
        coeff.append(mdf.params['timeAfterBirth'])
        se.append(SE["timeAfterBirth"])
        r2.append(results_R2s['Dep. Variable:'])
        adjustR2.append(results_R2s['Model:'])
        aic.append(results_R2s['No. Observations:']) 
        pvals.append(results_R2s['Date:'])
        tractTrack1.append(tract)
        nodeTrack1.append(node)
        coeffType.append(coeff_type)
        intercept.append(inter)
        r2.append(results_R2s['Dep. Variable:'])
        adjustR2.append(results_R2s['Model:'])
        aic.append(results_R2s['No. Observations:']) 
        pvals.append(results_R2s['Date:'])
        
print(mdf.summary())
df=pd.DataFrame(tractTrack1)
df.columns=['tract']
df.insert(1,"node",nodeTrack1)
df.insert(2,"coeffType",coeffType)
df.insert(3,"coeff",coeff)
df.insert(4,"se",se)
df.insert(5,"intercept",intercept)
df.insert(6,"r2",r2)
df.insert(7,"adjustR2",adjustR2)
df.insert(8,"aic",aic)
df.insert(9,"pvals",pvals)

df.to_csv('./outputData/LM_result_inUtero_exUtero_noNorm.csv')
df.head()
df.min(axis=0)
df.max(axis=0)
dfInUteroExUtero=df

#df.min(axis=0)
#df.max(axis=0)
#df.mean(axis=0)
#dfInUtero=df[df['coeffType']==1]
#dfInUtero.min(axis=0)
#dfInUtero.max(axis=0)
#dfInUtero.mean(axis=0)
#dfExUtero=df[df['coeffType']==2]
#dfExUtero.min(axis=0)
#dfExUtero.max(axis=0)
#dfExUtero.mean(axis=0)

dfInUteroExUtero.head()
dfScanAge.head()
dfExUtero=dfInUteroExUtero[dfInUteroExUtero['coeffType']==2]
dfM=pd.merge(dfExUtero,dfScanAge, on=["tract","node"], how="inner")
dfM.head()
aic=dfM['aic_y']-dfM['aic_x']

print(aic)
aic.mean(axis=0)
aic.min(axis=0)
aic.max(axis=0)

statsPerBundle=df.groupby('tract').mean()
statsPerBundle.head()
statsPerBundle.to_csv('./outputData/LM_result_inUtero_exUtero_noNorm_meanPerBundle.csv')

df=dfInUteroExUtero[dfInUteroExUtero['coeffType']==1]
statsPerBundle=df.groupby('tract').mean()
statsPerBundle.head()
statsPerBundle.to_csv('./outputData/LM_result_inUtero_exUtero_noNorm_meanPerBundleInUtero.csv')

df=dfInUteroExUtero[dfInUteroExUtero['coeffType']==2]
statsPerBundle=df.groupby('tract').mean()
statsPerBundle.head()
statsPerBundle.to_csv('./outputData/LM_result_inUtero_exUtero_noNorm_meanPerBundleExUtero.csv')



In [ ]:
#this creates fig 2 of the main manuscript
df=pd.read_csv('./outputData/LM_result_inUtero_exUtero_noNorm.csv');

coeffType1=df[df['coeffType']==1];
coeffType2=df[df['coeffType']==2];


fig, axes = plt.subplots(2,6,frameon=False,figsize=(25,10));
fig.tight_layout;
fig.subplots_adjust(wspace=0.3, hspace=0.9);

ct = 0;
for tractName in tractCount:
    
    ax=axes[tractPos[tracts[ct]]];    
    currentTract=df.query("tract == @tractName");
    coeffType1CurrentTract=coeffType1.query('tract == @tractName');
    coeffType2CurrentTract=coeffType2.query('tract == @tractName');
    
    sns.set(font_scale = 2);
    sns.set_style('white');
    fig1 = sns.lineplot(data=coeffType1CurrentTract, x='node', y='coeff', hue='coeffType',legend=False, dashes=[(2,2)], style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=2); #I have changed the palette colour from 'crest' to 'Blues' as my computer didn't recognise that colour
    fig1 = sns.lineplot(data=coeffType2CurrentTract, x='node', y='coeff', hue='coeffType' ,legend=False, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=2);
    fig1 = ax.fill_between(coeffType1CurrentTract["node"], coeffType1CurrentTract['coeff']-coeffType1CurrentTract['se'],coeffType1CurrentTract['coeff']+coeffType1CurrentTract['se'],color=[color_list_all[color_order[ct]]],alpha=0.25);
    fig1 = ax.fill_between(coeffType2CurrentTract["node"], coeffType2CurrentTract['coeff']-coeffType2CurrentTract['se'], coeffType2CurrentTract['coeff']+coeffType2CurrentTract['se'], color=[color_list_all[color_order[ct]]],alpha=0.25); 

    _= ax.set_title(tracts[ct], pad=25, fontsize=25);
    _ = ax.set_xlabel(x_labels[ct], labelpad = 15, fontsize=25);
    _=ax.set_ylim([0,0.16])
    _=ax.hlines(y=0, xmin=0, xmax=100, linewidth=1, color='black',linestyle='dashed')
    
    if ct==2 or ct==3 or ct==12 or ct==13 or ct==8:
        _=ax.set_ylabel("T1w/T2w slope", fontsize=25);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,-1.0,-0.5,0.0])
        #ax.set_yticklabels([-1.5,-1.0,-0.5,0.0],Fontsize=25)
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['top'].set_visible(False);
        _=line1=Line2D([],[],color=color_list_all[ct],linestyle='dashed');
        _=line2=Line2D([],[],color=color_list_all[ct],linestyle='-');
        
    else:
        _=ax.set_ylabel('', fontsize=14);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,0.0])
        _=ax.yaxis.set_visible(False);
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['left'].set_visible(False);
        _=ax.spines['top'].set_visible(False);

    ct = ct + 1;
    
axes[0, 3].axis("off")
sns.set_style('white');
plt.show();
fig.savefig('./figures/fig2_compareCoeffs.png', dpi=600)



In [ ]:
#This generates Fig 3 in the main manuscript
#sort matched data into gropus and plot developement
#group 1: preterm babies scaned at birth
#gruop 2: preterm babies scanned full-term
#group 3: full-term babies matched to preterm babies scanned full-term

age0=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==1]
age0['scan_age'].mean()
age0.shape
age1=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2]
age1['scan_age'].mean()
age1.shape
age2=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==3]
age2['scan_age'].mean()
age2.shape

sns.set(font_scale = 2)
sns.set_style('white');
matchedData=allDataMergedCleaned[allDataMergedCleaned['matched_groups']>0]

df= matchedData;

fig, axes = plt.subplots(2,6,frameon=False,figsize=(25,10));
fig.tight_layout;
fig.subplots_adjust(wspace=0.3, hspace=0.9);
sns.set(font_scale = 2);

ct = 0;
for tractName in tractCount:
    
    ax=axes[tractPos[tracts[ct]]];    
    currentTract=df.query("tractID == @tractName");
    sns.set_style('white');
    
    group=1
    currentGroup=currentTract.query("matched_groups == @group");
    fig1=sns.lineplot(data=currentGroup, x='nodeID', y='t1wt2w',hue='matched_groups',legend=False, style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    group=2
    currentGroup=currentTract.query("matched_groups == @group");
    fig1=sns.lineplot(data=currentGroup, x='nodeID', y='t1wt2w',hue='matched_groups',legend=False, dashes=[(3,3)], style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    group=3
    currentGroup=currentTract.query("matched_groups == @group");
    fig1=sns.lineplot(data=currentGroup, x='nodeID', y='t1wt2w',hue='matched_groups',legend=False, dashes=[(1,1)], style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4)
    
    _= ax.set_title(tracts[ct], pad=25, fontsize=25);
    _ = ax.set_xlabel(x_labels[ct], labelpad = 15, fontsize=25);
    _=ax.set_ylim([0.3,2.2])
    
    if ct==2 or ct==3 or ct==12 or ct==13 or ct==8:
        _=ax.set_ylabel("T1w/T2w", fontsize=25);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,-1.0,-0.5,0.0])
        #ax.set_yticklabels([-1.5,-1.0,-0.5,0.0],Fontsize=25)
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['top'].set_visible(False);
        _=line1=Line2D([],[],color=color_list_all[ct],linestyle='dashed');
        _=line2=Line2D([],[],color=color_list_all[ct],linestyle='-');
        #ax.legend([line1,line2],['BB','AB'],loc="upper left",borderpad=0.01,bbox_to_anchor=(0.15,0.2),ncol=2,framealpha=0.75,frameon=True,edgecolor=[1,1,1],prop={'size':20},fancybox=True,handlelength=1.5)
        
    else:
        _=ax.set_ylabel('', fontsize=14);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,0.0])
        _=ax.yaxis.set_visible(False);
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['left'].set_visible(False);
        _=ax.spines['top'].set_visible(False);

    ct = ct + 1;
    
sns.set_style('white');
axes[0, 3].axis("off");
plt.show();
fig.savefig('./figures/fig3_premi_protracted_developement_t1wt2w.png', dpi=600)

In [ ]:
#predict t1w/t2w in preterm from scanAge model 

dfLM=pd.read_csv('./outputData/LM_result_scanAge_noNorm.csv');
allDataCleanedNoLongi=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==0]
groupsMergedCleanedPTSL=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2]
tractCount=allDataCleanedNoLongi['tractID'].unique()
tractCount.shape

coeffType1=dfLM[dfLM['coeffType']==3];

dfLM.head(20)
groupsMergedCleanedPTSL.head()
t1wt2wPredicted=[]
t1wt2wMeassured=[]
t1wt2wPred=[]
tractCount=groupsMergedCleanedPTSL['tractID'].unique()
tractCount.shape

print(tractCount)
nodeCount=groupsMergedCleanedPTSL['nodeID'].unique()
nodeCount.shape
print(nodeCount)
subjectsCount=groupsMergedCleanedPTSL['subjectID'].unique()
subjectsCount.shape
print(subjectsCount)

for subject in subjectsCount:

    for tract in tractCount:
        
        for node in nodeCount:
            
            groupsMergedCleanedPTSLForStats=groupsMergedCleanedPTSL[(groupsMergedCleanedPTSL['subjectID']==subject) & (groupsMergedCleanedPTSL['tractID']==tract) & (groupsMergedCleanedPTSL['nodeID']==node)]
            groupsMergedCleanedPTSLForStats.reset_index(level=0, inplace=True)
            #print(groupsMergedCleanedPTSLForStats['R1'])
            
            coeffType1ForStats=coeffType1[(coeffType1['tract']==tract) & (coeffType1['node']==node)]
            coeffType1ForStats.reset_index(level=0, inplace=True)

            t1wt2wPred=groupsMergedCleanedPTSLForStats['scan_age']*coeffType1ForStats['coeff'] + coeffType1ForStats['intercept']
            t1wt2wPredicted.append(t1wt2wPred)

df=pd.DataFrame(t1wt2wPredicted)
df.columns=['t1wt2wPredictedScanAge']

df.to_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_scanAge.csv')



In [ ]:
#predict t1wt2w in preterm from inUtero exUTero model 

dfLM=pd.read_csv('./outputData/LM_result_inUtero_exUtero_noNorm.csv')
groupsMergedCleanedPTSL=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2]

coeffType1=dfLM[dfLM['coeffType']==1];
coeffType2=dfLM[dfLM['coeffType']==2];
dfLM.head(20)
groupsMergedCleanedPTSL.head()
t1wt2wPredicted=[]
t1wt2wPred=[]
t1wt2wMeassured=[]
tractCount=groupsMergedCleanedPTSL['tractID'].unique()
tractCount.shape
print(tractCount)

nodeCount=groupsMergedCleanedPTSL['nodeID'].unique()
nodeCount.shape
print(nodeCount)

subjectsCount=groupsMergedCleanedPTSL['subjectID'].unique()
subjectsCount.shape
print(subjectsCount)

for subject in subjectsCount:

    for tract in tractCount:
        
        for node in nodeCount:
            
            groupsMergedCleanedPTSLForStats=groupsMergedCleanedPTSL[(groupsMergedCleanedPTSL['subjectID']==subject) & (groupsMergedCleanedPTSL['tractID']==tract) & (groupsMergedCleanedPTSL['nodeID']==node)]
            groupsMergedCleanedPTSLForStats.reset_index(level=0, inplace=True)
            
            coeffType1ForStats=coeffType1[(coeffType1['tract']==tract) & (coeffType1['node']==node)]
            coeffType1ForStats.reset_index(level=0, inplace=True)
            
            coeffType2ForStats=coeffType2[(coeffType2['tract']==tract) & (coeffType2['node']==node)]
            coeffType2ForStats.reset_index(level=0, inplace=True)
            
            t1wt2wPred=groupsMergedCleanedPTSLForStats['birth_age']*coeffType1ForStats['coeff'] + groupsMergedCleanedPTSLForStats['timeAfterBirth']*coeffType2ForStats['coeff'] + coeffType2ForStats['intercept']
            t1wt2wPredicted.append(t1wt2wPred)
             
df=pd.DataFrame(t1wt2wPredicted)
df.columns=['t1wt2wPredictedexUT']
df.shape
df.head
df.tail

df.to_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_exUT.csv')



In [ ]:
#this produces fig 4a in the main manuscipt
dfPTSL=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2]
dfPTSL.reset_index(drop=True,inplace=True)
dfPredscanAge=pd.read_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_scanAge.csv');
dfPredexUT=pd.read_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_exUT.csv');

dfIn=pd.concat([dfPTSL,dfPredscanAge,dfPredexUT],axis=1)

dfMerged=dfIn.groupby(["tractID","nodeID"], as_index=False).mean(numeric_only=True)
dfMerged.head()

df= dfIn;


fig, axes = plt.subplots(2,6,frameon=False,figsize=(25,10));
fig.tight_layout;
fig.subplots_adjust(wspace=0.3, hspace=0.9);
sns.set(font_scale = 2);
sns.set_style('white');

ct = 0;
for tractName in tractCount:
    ax=axes[tractPos[tracts[ct]]];
    _=ax.set_ylim([0.3,2])
    currentTract=df.query("tractID == @tractName");
    sns.set_style('white');
    grey_colors=sns.color_palette("Greys", 8)
    
    if (ct % 2) == 0: 
        fig1=sns.lineplot(data=currentTract, x='nodeID', y='t1wt2wPredictedScanAge',hue=1, legend=False, dashes=[(3,3)], style=True, ax=ax, palette=[grey_colors[2]], linewidth=4)
        fig1=sns.lineplot(data=currentTract, x='nodeID', y='t1wt2wPredictedexUT',hue=1, legend=False, dashes=[(1,1)], style=True, ax=ax, palette=[grey_colors[6]], linewidth=4)
    else:
        fig1=sns.lineplot(data=currentTract, x='nodeID', y='t1wt2wPredictedScanAge',hue=1, legend=False, dashes=[(3,3)], style=True, ax=ax, palette=[grey_colors[3]], linewidth=4)
        fig1=sns.lineplot(data=currentTract, x='nodeID', y='t1wt2wPredictedexUT',hue=1, legend=False, dashes=[(1,1)], style=True, ax=ax, palette=[grey_colors[7]], linewidth=4)
    
    fig1=sns.lineplot(data=currentTract, x='nodeID', y='t1wt2w',legend=False, hue=1, style=True, ax=ax, palette=[color_list_all[color_order[ct]]], linewidth=4,alpha=0.65)
    
    _= ax.set_title(tracts[ct], pad=25, fontsize=25);
    _ = ax.set_xlabel(x_labels[ct], labelpad = 15, fontsize=25);
    
    if ct==2 or ct==3 or ct==12 or ct==13 or ct==8:
        _=ax.set_ylabel("T1w/T2w", fontsize=25);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,-1.0,-0.5,0.0])
        #ax.set_yticklabels([-1.5,-1.0,-0.5,0.0],Fontsize=25)
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['top'].set_visible(False);
        _=line1=Line2D([],[],color=color_list_all[ct],linestyle='dashed');
        _=line2=Line2D([],[],color=color_list_all[ct],linestyle='-');
        #ax.legend([line1,line2],['BB','AB'],loc="upper left",borderpad=0.01,bbox_to_anchor=(0.15,0.2),ncol=2,framealpha=0.75,frameon=True,edgecolor=[1,1,1],prop={'size':20},fancybox=True,handlelength=1.5)
        
    else:
        _=ax.set_ylabel('', fontsize=14);
        _=ax.set_xticks([0,50,100]);
        _=ax.set_xticklabels([0,50,100], fontsize=25);
        #ax.set_yticks([-1.5,0.0])
        _=ax.yaxis.set_visible(False);
        _=ax.spines['right'].set_visible(False);
        _=ax.spines['left'].set_visible(False);
        _=ax.spines['top'].set_visible(False);

    ct = ct + 1;
    
axes[0, 3].axis("off");
sns.set_style('white');
plt.show();
fig.savefig('figures/fig4a_predict_sampl2_from_sampl1.png', dpi=600)



In [ ]:
#this produces fig 4b in the main manuscipt
font = {'size'   : 12}
mpl.rc("font", **font)

fs = 16
dfPTSL=allDataMergedCleaned[allDataMergedCleaned['matched_groups']==2]
dfPTSL.reset_index(drop=True, inplace=True)
#dfPTSL.head()
dfPredscanAge=pd.read_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_scanAge.csv');
#dfPredscanAge.head()
dfPredexUT=pd.read_csv('./outputData/groupsMergedCleanedPTSL_predicted_t1wt2w_exUT.csv');
#dfPredexUT.head()
dfIn=pd.concat([dfPTSL,dfPredscanAge,dfPredexUT],axis=1)
#dfIn.head()
subjects = dfIn["subjectID"].unique()
#df=dfIn
r2_sa = []
r2_eu = []
ba = []
sa = []
for subject in subjects:
    subject_df = df[df["subjectID"] == subject][["t1wt2wPredictedScanAge", "t1wt2wPredictedexUT", "t1wt2w", "birth_age", "timeAfterBirth"]];
    r2_sa.append(r2_score(subject_df["t1wt2w"], subject_df["t1wt2wPredictedScanAge"]));
    r2_eu.append(r2_score(subject_df["t1wt2w"], subject_df["t1wt2wPredictedexUT"]));
    ba.append(subject_df["birth_age"].iloc[0]);
    sa.append(subject_df["timeAfterBirth"].iloc[0]);
np.mean(r2_sa)
np.mean(r2_eu)
mannwhitneyu(r2_sa, r2_eu)

In [ ]:
norm_ba = ba - min(ba);
norm_ba = norm_ba / max(norm_ba);
norm_sa = sa - min(sa);
norm_sa = norm_sa / max(norm_sa);
#print(r2_sa);
#print(subject_df["t1wt2w"]);
#print(subject_df["t1wt2wPredictedScanAge"]);
fig, ax = plt.subplots();
for ii in range(len(subjects)):
    _ = ax.plot([0, 1], [r2_sa[ii], r2_eu[ii]], 'ko-', alpha=0.5, color=cmap(norm_sa[ii]), linewidth=2, markersize=10);

_ = ax.set_xticks([0, 1]);
_ = ax.set_xlim([-0.5, 1.5]);
_=ax.tick_params(length=5, width=2, direction="out", reset=True);
_=ax.spines['top'].set_visible(False);
_=ax.spines['right'].set_visible(False);
ss = ax.spines['left'];
_=ss.set_linewidth(2);
ss = ax.spines['bottom'];
_=ss.set_linewidth(2);
_=ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0]);
_=ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1], fontsize=fs);
_=ax.set_ylabel("$R^2$", fontsize=fs+2);
_=ax.set_xlabel("model", fontsize=fs+2);
#ax.spines[['right', 'top']].set_visible(False)
_=ax.get_xaxis().tick_bottom();
_=ax.get_yaxis().tick_left();
_=ax.set_xticklabels(["scan age", "$\it{in~utero}$/$\it{ex~utero}$"], fontsize=fs);
_=fig.set_size_inches([4, 6]);
cbaxes = inset_axes(ax, width="3%", height="50%", loc=4);
cb = fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin=min(sa)-1, vmax=max(sa)+1), cmap=cmap), ax=ax, cax=cbaxes);
_= cb.ax.set_ylabel("Time since birth (weeks)", fontsize=fs);
_=cb.ax.set_yticklabels(cb.ax.get_yticklabels(), fontsize=fs);

fig.savefig('figures/fig4b_predict_sampl2_from_sampl1.png')